In [1]:
import pandas as pd
import numpy as np
from functools import partial
from scipy.stats import percentileofscore

In [2]:
df = pd.read_csv("../data/drugs_2019_20210323.csv")

In [3]:
df.columns

Index(['offender_id', 'dm_offender_race_ethnicity', 'dm_offender_sex',
       'dm_offender_age', 'cleared_except_id', 'arrest_type', 'agency',
       'crack_mass', 'cocaine_mass', 'heroin_mass', 'cannabis_mass',
       'meth_amphetamines_mass', 'other_drugs', 'criminal_act_count',
       'criminal_act', 'criminal_act_id', 'offense_count', 'offender_count',
       'location_category', 'drug_equipment_value'],
      dtype='object')

In [4]:
def conditional_quantile(data, input_column, output_column, condition=0):
    quantile_func = partial(np.quantile, q=[0.2, 0.4, 0.6, 0.8, 1])
    cond = data[input_column] > condition
    dc = data[cond]
    dnc = data[~cond]
    dc_q = quantile_func(dc[input_column])
    quantile_conv = lambda x: str(percentileofscore(dc_q, x[input_column], kind="strict"))
    dc[output_column] = df.apply(quantile_conv, axis=1)
    dnc[output_column] = "Zero"
    return dc.append(dnc)[output_column]

In [5]:
cocaine = conditional_quantile(df, "cocaine_mass", "cocaine_quantile")
heroin = conditional_quantile(df, "heroin_mass", "heroin_quantile")
cannabis = conditional_quantile(df, "cannabis_mass", "cannabis_quantile")
meth_amphetamines = conditional_quantile(df, "meth_amphetamines_mass", "meth_amphetamines_quantile")
crack = conditional_quantile(df, "crack_mass", "crack_quantile")

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [6]:
df = pd.concat([df, cocaine, heroin, cannabis, meth_amphetamines, crack], axis=1)

In [7]:
to_drop = set(['crack_mass', 'cocaine_mass', 'heroin_mass', 'cannabis_mass', 'meth_amphetamines_mass', 'offender_id']) & set(df.columns)
df = df.drop(to_drop, axis=1)

In [8]:
df.crack_quantile.value_counts()

Zero    622696
20.0      8525
60.0      6426
0.0       6356
80.0      6232
40.0      4237
Name: crack_quantile, dtype: int64

In [9]:
df.nunique()

dm_offender_race_ethnicity       5
dm_offender_sex                  3
dm_offender_age                  7
cleared_except_id                6
arrest_type                      4
agency                        6327
other_drugs                      5
criminal_act_count               6
criminal_act                     6
criminal_act_id                  7
offense_count                    2
offender_count                   4
location_category                6
drug_equipment_value             3
cocaine_quantile                 6
heroin_quantile                  6
cannabis_quantile                6
meth_amphetamines_quantile       6
crack_quantile                   6
dtype: int64

In [10]:
df.to_csv("../data/drugs_2019_20210324.csv", index=False)